https://github.com/cjhutto/vaderSentiment
    
Utilisation du fichier vader_lexicon.txt pour calculer la pos/neg/neu d'un tweet.
Ce fichier a été crée "à la main" en demandant à plusieurs personnes de donner leur sentiment autour des mots proposés.
Avantage: bonne accuracy des résultats
Inconvénient: ne connait pas beaucoup de mot: classe beaucoup de tweet en neutre

In [49]:
from textblob import TextBlob
import sys
import re

def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(clean_tweet(tweet))
        # set sentiment
        return analysis.sentiment.polarity
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
def getDataLines(file_path):
    f = open(file_path, "r")
    lines = f.readlines()
    f.close
    return lines

def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(clean_tweet(tweet))
        # set sentiment
        return analysis.sentiment.polarity
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

# Train Data

In [12]:
path = "../data/michigan/training.txt"
lines = getDataLines(path)

count  = 0
acc = 0

for l in lines:
    l = l.split("\t")
    l[1] = l[1][:-1]
    feeling = get_tweet_sentiment(l[1])
    if (feeling != 0):
        count = count + 1
        if ((feeling > 0 and int(l[0]) == 1) or (feeling < 0 and int(l[0]) == 0)):
            acc = acc + 1
    
print(acc/count)

0.8401940835811551


# Train Data Positif

In [13]:
path = "../data/michigan/training.txt"
lines = getDataLines(path)

count  = 0
acc = 0

for l in lines:
    l = l.split("\t")
    l[1] = l[1][:-1]
    feeling = get_tweet_sentiment(l[1])
    if (feeling != 0 and int(l[0]) == 1):
        count = count + 1
        if (feeling > 0):
            acc = acc + 1
    
print(acc/count)

0.8372452927521279


# Train Data Negatif

In [14]:
path = "../data/michigan/training.txt"
lines = getDataLines(path)

count  = 0
acc = 0

for l in lines:
    l = l.split("\t")
    l[1] = l[1][:-1]
    feeling = get_tweet_sentiment(l[1])
    if (feeling != 0 and int(l[0]) == 0):
        count = count + 1
        if (feeling < 0):
            acc = acc + 1
    
print(acc/count)

0.8447452229299363


# Test Data

In [15]:
path = "../data/michigan/testdata.txt"
lines = getDataLines(path)

neg = 0
pos = 0

for l in lines:
    feeling = get_tweet_sentiment(l)
    if (feeling != 0):
        count = count + 1
        if (feeling > 0):
            pos += 1
        else:
            neg += 1
            
    
print("Pos:", pos)
print("Neg:", neg)

Pos: 17581
Neg: 10599


In [67]:
from vaderSentiment import sentiment
path = "../data/michigan/training.txt"
lines = getDataLines(path)


count = 0
acc = 0
n = 0
for l in lines:
    l = l.split("\t")
    l[1] = l[1][:-1]
    sent = sentiment(l[1])
    pos = float(sent['pos'])
    neg = float(sent['neg'])
    neu = float(sent['neu'])
    if (neu != 1):
        count += 1
        if ((int(l[0]) == 1 and pos > neg) or (int(l[0]) == 0 and neg > pos)):
            acc += 1
    else:
        feel = get_tweet_sentiment(l[1])
        if (feel != 0):
            count += 1
            if ((int(l[0]) == 1 and feel > 0) or (int(l[0]) == 0 and feel < 0)):
                acc += 1
        else:
            n += 1

    
print(acc/count, "% accuracy")
print(count, "tweets étudiés")
print(n, "tweets neutres")
print(count/len(lines), "% des tweets utilisés")

0.901040903093388 % accuracy
6821 tweets étudiés
265 tweets neutres
0.9626023144228055 % des tweets utilisés


L'accuracy est de 100% dans le cas où on filtre les cas où pos ou neg est > 0.5
Si on filtre les neutre qui le sont à 1 alors on obtient une accuracy de 90%
Dans le cas normal, accuracy est de 80%

Si on utilise textblob dans le cas où le tweet est considéré comme neutre, alors on obtient 86% d'accuracy.
Si on ignore encore les tweet considéré comme neutre par vader ET textblob alors on obtient un accuracy de 90%